In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error, log_loss
import sklearn
import os
from matplotlib.pyplot import hist
from psmpy import PsmPy
from psmpy.functions import cohenD
from psmpy.plotting import *
# import more functions or modules if you need them !!

In [2]:
# set random seed for numpy
RANDOM_SEED=42
np.random.seed(RANDOM_SEED)

In [4]:
# ATT and ATE AIPTW
def att_aiptw(Q0, Q1, g, A, Y, prob_t=None):
    """
    Double ML estimator for the ATT
    This uses the ATT specific scores, see equation 3.9 of https://www.econstor.eu/bitstream/10419/149795/1/869216953.pdf
    Return: aiptw of ATE and its standard error
    """
    
    # number of observations
    n = Y.shape[0]
    
    # estimate marginal probability of treatment
    if prob_t is None:
        prob_t = A.mean() 
    
    # att aiptw
    tau_hat = (A*(Y-Q0) - (1-A)*(g/(1-g))*(Y-Q0)).mean()/ prob_t
  
    # influence curve and standard error of aiptw
    phi = (A*(Y-Q0) - (1-A)*(g/(1-g))*(Y-Q0) - tau_hat*A) / prob_t
    std_hat = np.std(phi) / np.sqrt(n)

    return tau_hat, std_hat

def ate_aiptw(Q0, Q1, g, A, Y, prob_t=None):
    """
    Double ML estimator for the ATE
    Return: aiptw of ATE and its standard error
    """
    # number of observations
    n = Y.shape[0]
    
    # ate aiptw
    tau_hat = (Q1 - Q0 + A*(Y-Q1)/g - (1-A)*(Y-Q0)/(1-g)).mean()
  
    # influence curve and standard error of aiptw
    phi = Q1 - Q0 + A*(Y-Q1)/g - (1-A)*(Y-Q0)/(1-g) - tau_hat   
    std_hat = np.std(phi) / np.sqrt(n)

    return tau_hat, std_hat

In [5]:
# Conditional outcome models (Q models)
def make_linear_Q_model():
    ''' A function that returns a linear q model for later use in k-folding'''
    return LinearRegression()

def make_Q_model(output_type:str):
    ''' A function that returns a general ML q model for later use in k-folding'''
    if output_type == 'binary':
        return RandomForestClassifier(random_state=RANDOM_SEED, n_estimators=500, max_depth=None)
    return RandomForestRegressor(random_state=RANDOM_SEED, n_estimators=500, max_depth=None)
# One example: RandomForestRegressor(random_state=RANDOM_SEED, n_estimators=500, max_depth=None)

In [6]:
# Propensity score models (g models)
def make_g_model():
    ''' A function that returns a g model for computing propensity scores'''
    return RandomForestClassifier(n_estimators=100, max_depth=5)
# One example: RandomForestClassifier(n_estimators=100, max_depth=5)

In [7]:
# Functions for K-fold cross-fitting
def treatment_k_fold_fit_and_predict(make_model, X:pd.DataFrame, A:np.array, n_splits:int):
    '''
    Implements K fold cross-fitting for the model predicting the treatment A. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns an array containing the predictions  

    Args:
    model: function that returns sklearn model (which implements fit and predict_prob)
    X: dataframe of variables to adjust for
    A: array of treatments
    n_splits: number of splits to use
    '''

    predictions = np.full_like(A, np.nan, dtype=float)
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    
    for train_index, test_index in kf.split(X, A):
        X_train = X.loc[train_index]
        A_train = A.loc[train_index]
        g = make_model()
        g.fit(X_train, A_train)

        # get predictions for split
        predictions[test_index] = g.predict_proba(X.loc[test_index])[:, 1]
    
    # sanity check that overlap holds
    assert np.isnan(predictions).sum() == 0
    return predictions

def outcome_k_fold_fit_and_predict(make_model, X:pd.DataFrame, y:np.array, A:np.array, n_splits:int, output_type:str):
    '''
    Implements K fold cross-fitting for the model predicting the outcome Y. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns two arrays containing the predictions for all units untreated, all units treated  

    Args:
    model: function that returns sklearn model (that implements fit and either predict_prob or predict)
    X: dataframe of variables to adjust for
    y: array of outcomes
    A: array of treatments
    n_splits: number of splits to use
    output_type: type of outcome, "binary" or "continuous"
    '''

    predictions0 = np.full_like(A, np.nan, dtype=float)
    predictions1 = np.full_like(y, np.nan, dtype=float)
    if output_type == 'binary':
        kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    elif output_type == 'continuous':
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)

    # include the treatment as input feature
    X_w_treatment = X.copy()
    X_w_treatment["A"] = A

    # for predicting effect under treatment / control status for each data point 
    X0 = X_w_treatment.copy()
    X0["A"] = 0
    X1 = X_w_treatment.copy()
    X1["A"] = 1

    
    for train_index, test_index in kf.split(X_w_treatment, y):
        X_train = X_w_treatment.loc[train_index]
        y_train = y.loc[train_index]
        q = make_model(output_type)
        q.fit(X_train, y_train)

        if output_type =='binary':
            predictions0[test_index] = q.predict_proba(X0.loc[test_index])[:, 1]
            predictions1[test_index] = q.predict_proba(X1.loc[test_index])[:, 1]
        elif output_type == 'continuous':
            predictions0[test_index] = q.predict(X0.loc[test_index])
            predictions1[test_index] = q.predict(X1.loc[test_index])

    assert np.isnan(predictions0).sum() == 0
    assert np.isnan(predictions1).sum() == 0
    return predictions0, predictions1

In [8]:
def fit_and_run_model(df, outcome:str, treatment:str, confounders:list, make_g_model,
                      make_Q_model, n_splits=5, output_type='binary', ate=True):
    '''
    Function that creates a g, q, and aiptw model based on the 
    given inputs
    
    Inputs: df (pandas df) - the dataframe the variables are contained in
            outcome (str) - the outcome variable
            treatment (str) - the treatment variable
            confounders (lst) - a list of the confounding variables
            make_g_model - the make_g_model function
            make_Q_model - the make_Q_model function
            n_splits (int) - number of splits for the model
            output_type (str) - the desired output type, either binary or continous
            ate (bool) - whether to use ate or alternative att
    
    Returns: tau_hat - the tau hat estimator for the average treatment effect
             std of tau_hat - the standard deviation for the tau_hat estimator
    '''
    df = df.replace({outcome: .00001}, 0)
    df = df[[outcome] + confounders + [treatment]]
    df = df.dropna().reset_index()
    print('Running models for treatment {} and outcome {} on {} samples'.format(treatment, outcome, len(df)))
    outcome = df[outcome]
    confounders = df[confounders]
    treatment = df[treatment]
    
    g = treatment_k_fold_fit_and_predict(make_g_model, X=confounders, A=treatment, n_splits=n_splits)
    
    if min(g) < .01:
        print('\nWARNING:\n Some propensity scores are very small,\n which could '
              'lead to an inflated AIPTW.\n Minimum score = ', min(g))
    if max(g) > .99:
        print('\nWARNING:\n Some propensity scores are very large,\n which could '
              'lead to an inflated AIPTW.\n Maximum score = ', max(g))
    print('G Model has been fit')

    Q0_ml, Q1_ml = outcome_k_fold_fit_and_predict(make_Q_model, X=confounders, y=outcome, A=treatment, \
                                                  n_splits=n_splits, output_type=output_type)
    
    print('Q model has been fit')
    data_and_nuisance_estimates_ml = pd.DataFrame({'g': g, 'Q0': Q0_ml, 'Q1': Q1_ml, 'A': treatment, 'Y': outcome})
    
    # ate aiptw
    if ate:
        tau_hat, std_hat = ate_aiptw(**data_and_nuisance_estimates_ml)
    else: 
        tau_hat, std_hat = att_aiptw(**data_and_nuisance_estimates_ml)
    print('AIPTW model has been fit. Returning \u03C4 hat and its standard deviation')
    print('\u03C4 hat = {} and std = {}\n'.format(round(tau_hat, 5), round(std_hat, 5)))
    return tau_hat, std_hat

In [14]:
df = pd.read_stata('/Users/zwanran/Desktop/STAT27420/Final/STAT27420-final-code/Data/data/aspo_full_matching.dta')

In [20]:
model_vars = ['onset2COWCS',
              'success',
              'wildcat_diff',
              'wildcat_diff_binary',
              'valoilres_diff',
              'ecgrowth', 
              'popdens_diff', 
              'democracy_diff', 
              'numcode',
              'logmountain',
              'incidence2COW',
              'africa',
              'southam',
              'asia',
              'oceania',
              'year']
df = df[model_vars]
df = df.dropna()
df['index'] = np.arange(len(df))

In [23]:
psm = PsmPy(df, treatment='wildcat_diff_binary', indx='index', exclude = ['year', 'numcode', 'success', 'wildcat_diff', 'onset2COWCS'])
psm.logistic_ps(balance = True)
psm.predicted_data

,index,valoilres_diff,ecgrowth,popdens_diff,democracy_diff,logmountain,incidence2COW,africa,southam,asia,oceania,propensity_score,propensity_logit,wildcat_diff_binary
0,11,-0.001408,0.038898,0.000306,0.000000,0.027537,0.00001,1.00000,0.00001,0.00001,0.00001,0.479142,-0.083482,1
1,12,-0.000054,0.034214,0.000302,0.000000,0.027537,0.00001,1.00000,0.00001,0.00001,0.00001,0.477765,-0.089000,1
2,33,0.002051,0.056571,0.000210,-0.125000,0.022721,0.00001,1.00000,0.00001,0.00001,0.00001,0.482472,-0.070141,1
3,35,0.000188,0.120336,0.000208,0.142857,0.022721,0.00001,1.00000,0.00001,0.00001,0.00001,0.509772,0.039091,1
4,43,-0.001942,0.093678,0.000049,0.000000,0.032426,0.00001,0.00001,0.00001,1.00000,0.00001,0.470816,-0.116871,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2742,2740,-0.001936,0.046573,0.000322,0.000000,0.033032,0.00001,0.00001,0.00001,1.00000,0.00001,0.368021,-0.540715,0
2743,2741,-0.005182,0.015156,0.000328,0.000000,0.033032,0.00001,0.00001,0.00001,1.00000,0.00001,0.358615,-0.581383,0
2744,2742,0.002095,0.004042,0.000333,0.000000,0.033032,0.00001,0.00001,0.00001,1.00000,0.00001,0.356072,-0.592452,0
2745,2744,-0.002969,0.003624,0.000337,0.000000,0.033032,0.00001,0.00001,0.00001,1.00000,0.00001,0.355495,-0.594971,0


In [24]:
psm.knn_matched(matcher='propensity_logit', replacement=False, caliper=None)
# version 2:
# psm.knn_matched_12n(matcher='propensity_logit', how_many=2)

/Users/zwanran/opt/anaconda3/lib/python3.9/site-packages/psmpy/psmpy.py:346: UserWarning: Some values do not have a match
  warnings.warn('Some values do not have a match')


In [ ]:
treatment_lst = ['wildcat_diff']
outcome_lst = ['onset2COWCS']
confounders = [x for x in model_vars if x not in (outcome + treatment)]

In [ ]:
df_dict = {'model': [], 'treatment': [], 'outcome': [], 'tau_hat': [], 'tau_std': []}
for treat in treatment_lst:
    for out in outcome_lst:
        if len(oil_df[out].value_counts()) == 2:
            output = 'binary'
        else:
            output = 'continuous'
        tau_hat, tau_std = fit_and_run_model(oil_df, out, treat, confounders, make_g_model, make_Q_model, output_type=output)
        df_dict['model'].append('ate')
        df_dict['treatment'].append(treat)
        df_dict['outcome'].append(out)
        df_dict['tau_hat'].append(tau_hat)
        df_dict['tau_std'].append(tau_std)

ate_results_df = pd.DataFrame(data=df_dict)

In [ ]:
df_dict = {'model': [], 'treatment': [], 'outcome': [], 'tau_hat': [], 'tau_std': []}
for treat in treatment_lst:
    for out in outcome_lst:
        if len(oil_df[out].value_counts()) == 2:
            output = 'binary'
        else:
            output = 'continuous'
        tau_hat, tau_std = fit_and_run_model(oil_df, out, treat, confounders, make_g_model, make_Q_model, output_type=output, ate=False)
        df_dict['model'].append('att')
        df_dict['treatment'].append(treat)
        df_dict['outcome'].append(out)
        df_dict['tau_hat'].append(tau_hat)
        df_dict['tau_std'].append(tau_std)

att_results_df = pd.DataFrame(data=df_dict)

In [ ]:
results_df = pd.concat([ate_results_df, att_results_df])
results_df